In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras import backend as K
from sklearn.model_selection import KFold

In [15]:
batch_size = 128
epochs = 1
IMG_HEIGHT = 224
IMG_WIDTH = 224
total_train = 22400
epochs_fine = 10
num_folds = 5
IMG_SHAPE = (IMG_HEIGHT, IMG_WIDTH, 3)
K.clear_session()

In [4]:
# Leemos datos y etiquetas de memoria
x_array = np.load('datos.npy')
y_array = np.load('etiquetas.npy')

In [16]:
# Creamos la clase que nos va a dividir los datos
kfold = KFold(n_splits=num_folds, shuffle=True)

# Variable para almacenar el número de fold por el que vamos
num_fold = 1

# Variables donde almacenar la accuracy y la pérdida de cada fold
accuracies = []
losses = []

# Realizamos la validacion cruzada
for train, test in kfold.split(x_array, y_array): 
    # Obtenemos datos y etiquetas de entrenamiento
    datos_train = x_array[train]
    etiquetas_train = y_array[train]
    
    # Obtenemos datos y etiquetas de test
    datos_test = x_array[test]
    etiquetas_test = y_array[test]

    # Create el modelo base
    base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                                   include_top=False,
                                                   weights='imagenet')
    base_model.trainable = False
    
    global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
    prediction_layer = tf.keras.layers.Dense(28, activation='softmax')

    model = tf.keras.Sequential([
      base_model,
      global_average_layer,
      prediction_layer
    ])
    
    model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

    print('------------------------------------------------------------------')
    print(f'Ejecutando fold {num_fold}')
    
    print(f'Ejecutando transfer learning')
    
    history = model.fit(
        x=datos_train, 
        y=etiquetas_train, 
        batch_size=batch_size,
        epochs=epochs)
    
    base_model.trainable = True

    model.compile(
        optimizer=tf.keras.optimizers.Adam(1e-5),  
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True,),
        metrics=['accuracy']
    )
    
    print(f'Ejecutando fine tuning')

    history = model.fit(
        x=datos_train, 
        y=etiquetas_train, 
        batch_size=batch_size,
        epochs=epochs_fine)

    # Evaluamos el modelo con los datos de test
    scores = model.evaluate(datos_test, etiquetas_test, verbose=0)
    
    print(f'fold={num_fold}: perdida={scores[0]} - accuracy={scores[1]*100}%')
    
    accuracies.append(scores[1] * 100)
    losses.append(scores[0])
    
    K.clear_session()

    # Pasamos el siguiente fold
    num_fold = num_fold + 1

------------------------------------------------------------------
Ejecutando fold 1
Ejecutando transfer learning
175/175 [==============================] - 19s 97ms/step - loss: 1.9994 - accuracy: 0.4495
Ejecutando fine tuning
Epoch 1/10
175/175 [==============================] - 99s 546ms/step - loss: 3.2021 - accuracy: 0.2897
Epoch 2/10
175/175 [==============================] - 93s 532ms/step - loss: 0.9150 - accuracy: 0.7566
Epoch 3/10
175/175 [==============================] - 97s 556ms/step - loss: 0.5485 - accuracy: 0.8554
Epoch 4/10
175/175 [==============================] - 97s 556ms/step - loss: 0.4042 - accuracy: 0.8868
Epoch 5/10
175/175 [==============================] - 94s 537ms/step - loss: 0.3213 - accuracy: 0.9102
Epoch 6/10
175/175 [==============================] - 95s 544ms/step - loss: 0.2555 - accuracy: 0.9243
Epoch 7/10
175/175 [==============================] - 97s 552ms/step - loss: 0.2149 - accuracy: 0.9383
Epoch 8/10
175/175 [==============================]

In [17]:
# Mostramos los resultados
for i in range(len(accuracies)):
    print('-------------------------------------------------------------------')
    print(f'Fold {i+1}: Loss={losses[i]} - Accuracy={accuracies[i]}%')
    
print('-------------------------------------------------------------------')
print(f'Final: Loss={np.mean(losses)} - Accuracy={np.mean(accuracies)}% (+- {np.std(accuracies)})')

-------------------------------------------------------------------
Fold 1: Loss=2.6016950607299805 - Accuracy=42.98214316368103%
-------------------------------------------------------------------
Fold 2: Loss=2.2844464778900146 - Accuracy=43.19642782211304%
-------------------------------------------------------------------
Fold 3: Loss=2.343014717102051 - Accuracy=40.69642722606659%
-------------------------------------------------------------------
Fold 4: Loss=2.309835910797119 - Accuracy=40.48214256763458%
-------------------------------------------------------------------
Fold 5: Loss=2.38970947265625 - Accuracy=42.38256812095642%
-------------------------------------------------------------------
Final: Loss=2.385740327835083 - Accuracy=41.94794178009033% (+- 1.142982586506918)


In [ ]:
model_name = 'modeloFinalFineTuning.h5'

In [16]:
model.save(model_name)

In [6]:
new_model = tf.keras.models.load_model(model_name)

In [18]:
pred = new_model.predict(x_array)

In [27]:
pred[0]

array([3.52496536e-05, 4.26581028e-05, 4.83288964e-10, 1.47379169e-05,
       2.89587064e-07, 1.05237192e-03, 3.27298039e-04, 1.22638352e-01,
       2.08580673e-06, 4.47910224e-06, 8.05806984e-12, 1.22808362e-03,
       8.22458730e-07, 1.09876972e-04, 8.63652458e-05, 8.70499253e-01,
       4.27346735e-04, 1.09955067e-06, 4.80066582e-08, 1.93857122e-05,
       1.19256205e-08, 5.21227194e-04, 2.02319073e-03, 6.92204367e-06,
       1.17451680e-04, 6.79058721e-04, 1.11104591e-05, 1.51252694e-04],
      dtype=float32)

In [36]:
num = 5
np.argmax(pred[num]) == y_array[num]

True

In [20]:
new_scores = new_model.evaluate(datos_test, etiquetas_test, verbose=0)
print(f'Score: {new_model.metrics_names[0]} of {new_scores[0]}; {new_model.metrics_names[1]} of {new_scores[1]*100}%')

Score: loss of 0.09432641416788101; accuracy of 96.910160779953%
